# Assignment 1

This assignment will involve the creation of a spellchecking system and an evaluation of its performance. You may use the code snippets provided in Python for completing this or you may use the programming language or environment of your choice

Please start by downloading the corpus `holbrook.txt` from Blackboard

The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .
    
For the purpose of this assignment you do not need to separate these words, but instead you may treat them like a single token.

## Task 1 (10 Marks)

Write a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes.

Then split your data into a test set of 100 lines and a training set.

In [1]:
import nltk 
#Importing the text file
filename = 'C:/####/holbrook.txt'

#Reading the file
textFile=open(filename,"r")
data = textFile.readlines()

#Counting the total number of lines
num_lines = sum(1 for line in open(filename))

#l gives us the total lines
l=num_lines-1

#Creating an empty list 
sentenceList=[]

#Filling the list with data
for i in range(0,l):
    sentenceList.append(nltk.word_tokenize(data[i]))

#Creating an empty list which would contain the original sentences
ORG=[]

#Creating an empty list which would contain the corrected sentences
COR=[]

In [2]:
   
from nltk import word_tokenize
import more_itertools as mit
import itertools
from itertools import chain

#Creating an empty list which would contain the location of misspelled words
INDEX=[]

#splitFunction breaks the data into correctly and incorrectly spelled sentences
def splitFunction(tokens):
    #checking the no. of tokens
    tokens_list = sum(1 for line in tokens)
    slashappender=[]
    original=[]
    corrected=[]
    index=[]
    #splitting tokens on the basis of |
    for i in range(0,tokens_list-1):
        slashappender.append(tokens[i].split("|"))
    for i in range(0,tokens_list-1):
        #if the token does not contain a slash
        #add the word to the original and corrected lists
        if(sum(1 for line in slashappender[i])==1):
            string1=slashappender[i]
            original.append(string1)
            corrected.append(string1)
        else:
            sublistlength=sum(1 for line in slashappender[i])
            for j in range(0,sublistlength-1):
                #word before slash
                string1=slashappender[i][j]
                #word after slash
                string2=slashappender[i][j+1]
                #add the tokens as list to the respective lists
                original.append(string1.split())
                corrected.append(string2.split())
                #add the index of the misspelled word/s
                index.append(i)
    ORG.append(original)
    COR.append(corrected)
    INDEX.append(index)

#splitFunction(tokens)
#splitFunction(tokens2)



In [3]:
#This loop passes each of the sentences in the list to the splitFunction
for i in range(0,l):
    splitFunction(sentenceList[i])

    
#This nested loop turns each word in the ORG and COR into lower case
for i in range(0,l):
    for j in range(0,len(ORG[i])):
        if(len(COR[i][j])!=0 and len(ORG[i][j])!=0):
            ORG[i][j][0]=ORG[i][j][0].lower()
            COR[i][j][0]=COR[i][j][0].lower()
        


In [4]:
from collections import defaultdict
#Making a dictionary
dictionary = []
dictionary = defaultdict(list)
#This contains information of ORG, COR, and INDEX in each of its index
for i in range(0,len(ORG)):
    dictionary[i].append(ORG[i])
    dictionary[i].append(COR[i])
    dictionary[i].append(INDEX[i])


In [5]:
# Creating training and test datasets
testingData = []
testingData = defaultdict(list)
#Contains first 100 dictionary items
for i in range(0,len(dictionary)-1116):
    testingData[i]=dictionary[i]
    

trainingData = []
trainingData = defaultdict(list)
k = 0
#Contains all the dictionary items save the first hundred
for i in range(99,len(dictionary)):
    trainingData[k]=dictionary[i]
    k = k+1
    


## **Task 2** (10 Marks): 
Calculate the frequency (number of occurrences), *ignoring case*, of all words and bigrams (sequences of two words) from the corrected *training* sentences:

In [6]:
from nltk.util import ngrams
def unigram(word):
    # Write your code here.
    #This function takes in a word 
    #and checks the whole corrected training corpus for its presence.
    count=0
    for i in range(0,len(trainingData)):
        for j in range(0,len(trainingData[i][1])):
            #Checks if the corpus contains a word
            #And if so, whether it matches the word
            #And if so, increments the count by 1
            if(len(trainingData[i][1][j])!=0 and word==trainingData[i][1][j][0]):
                count=count+1
    return count  
    

def bigram(words):
    # Write your code here.
    count=0
    for i in range(0,len(trainingData)):
        for j in range(0,len(trainingData[i][1])):
            #This checks if the corpus not just contains a word
            #But also a word following the word
            #If a match occurs, count is incremented by 1.
            if((j+1)<len(trainingData[i][1]) and len(trainingData[i][1][j])!=0 and len(trainingData[i][1][j+1])!=0 and words==(trainingData[i][1][j][0]+" "+trainingData[i][1][j+1][0]) ):
                count=count+1
    return count

# Test your code with the following

print(unigram('me'))
#assert(bigram("my mother")==17)
print(bigram("my mother"))

83
17


In [31]:
# Test your code with the following
assert(unigram('me')==83)
assert(bigram('my mother')==17)
assert(bigram('you did')==1)

## **Task 3** (15 Marks): 
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [7]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("hello", "hi"))

4


Write a function that calculates all words with *minimal* edit distance to the misspelled word. You should do this as follows

1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value

*Do not implement edit distance, use the built-in NLTK function `edit_distance`*

In [8]:
from itertools import chain
#An empty list
bagofWords=[]
#This nested loop appends to the bagofWords
#All the words that are present in the COR 
for i in range(0,len(COR)-1):
    for j in range(0,len(COR[i])):
        bagofWords.append(COR[i][j])

bagofWords=list(chain.from_iterable(bagofWords))

#From this, we get the unique correct words present in the corpus
uniqueWords=set(bagofWords)

#Turning it into a list
uniqueWords=list(uniqueWords)



In [9]:
from nltk.metrics.distance import edit_distance
def get_candidates(token):
    # Write your code here.
    #List containing edit distances
    editDistanceList=[]
    #list containg possible candidates for right spelling
    candidates=[]
    for i in range(0,len(uniqueWords)):
        editDistanceList.append(edit_distance(token,uniqueWords[i]))
    #Getting the minimum value for edit distance
    minimalValue=min(editDistanceList)
    for i in range(0,len(uniqueWords)):
        if(edit_distance(token,uniqueWords[i])==minimalValue):
            #appending the word/s with the minimum edit distances
            candidates.append(uniqueWords[i])
    return candidates
        
# Test your code as follows
get_candidates("minde") 

['mind', 'mine']

In [32]:
assert get_candidates("minde") == ['mind', 'mine']

## Task 4 (15 Marks):

Write a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system should scan the sentence for words that are not in the dictionary and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *bigram probability*. That is the candidate should be selected using the previous and following word in a bigram language model. Thus, if the $i$th word in a sentence is misspelled we should use the following to rank candidates:

$$p(w_{i+1}|w_i) p(w_i|w_{i-1})$$

For the first and last word of the sentence use only the conditional probabilities that exist.


In [10]:
import nltk
from itertools import chain
import math

# make a dictionary of all correct words from holbrook
dictionary1 = []
for i in range(len(COR)):
    dictionary1.append([x for x in COR[i]])

dictionary1 = list(chain.from_iterable(dictionary1))

#This function gives the correct sentence based on conditional probability
def correct(sentence):
    correctSentence = []
    for i in range(len(sentence)):
        #if sentence in dictionary1, then append
        if(sentence[i] in dictionary1):
            correctSentence.append(sentence[i])
        else:
            #get candidates for that token
            get_cand = list((get_candidates(sentence[i])))
            if(i < (len(sentence)-1) and i > 0):
                #get the words before and after the candidate word
                prefix = sentence[i-1]
                suffix = sentence[i+1]
                #the following list will contain conditional probabilities
                conditional_probability = []
                for a in range(0,len(get_cand)):
                    #Appends the conditional probability
                    conditional_probability.append(((bigram(prefix + " " + get_cand[a]))/(dictionary1.count(prefix)+0.1))*((bigram(get_cand[a] + " " + suffix))/(dictionary1.count(get_cand[a])+0.1)))
                #Gets the index of the maximum probability value
                highest_prob_ind = conditional_probability.index(max(conditional_probability))
                #get the candidate with the corresponding maximum highest probability 
                #and append to the correct Sentence
                correctSentence.append(get_cand[highest_prob_ind])
            else:
                #if the first word in the sentence is the wrong word 
                index = sentence.index(sentence[i])
                #checks if the index is 0 and the sentence consists of more than 1 word
                if(index == 0 and len(sentence)>1):
                    suffix = sentence[i+1]
                    conditional_probability = []
                    for a in range(0,len(get_cand)):
                        #calculates the probability
                        conditional_probability.append((bigram(get_cand[a] + " " + suffix))/(dictionary1.count(get_cand[a])+0.1))
                    
                    #appends the word with maximum probability
                    correctSentence.append(get_cand[conditional_probability.index(max(conditional_probability))])
                #this checks for the last word in the sentence being the wrong word
                elif(len(sentence)>1):
                    prefix = sentence[i-1]
                    cdf = []
                    for a in range(0,len(get_cand)):
                        cdf.append((bigram(prefix + " " + get_cand[a]))/(dictionary1.count(prefix)+0.1 ))
                    #appends the word with maximum probability
                    correctSentence.append(get_cand[cdf.index(max(cdf))])
                else:
                    #If the sentence consists of only 1 word
                    #This appends the candidate occuring first in the list
                    x=list((get_candidates(sentence[i])))[0]
                    correctSentence.append(x)
      #returns the correct sentence      
    return correctSentence






        

In [35]:
from itertools import chain
#Note: arguments passed must be in lower case
assert(correct(["this","whitr","cat"]) == ['this','white','cat'])

## **Task 5** (10 Marks): 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

In [18]:
def accuracy(test):
    count=0
    totalCount=0
    for i in range(len(dictionary)):
        for j in range(len(dictionary[i][2])):
            if(len(dictionary[i][2])> 0 and i<99):
                m=correct(list(chain.from_iterable(test[i][0])))
                if(m[j]==dictionary[i][1][j][0]):
                    count=count+1
                    totalCount=totalCount+1
                else:
                    totalCount=totalCount+1
    return(count/totalCount)

print(accuracy(testingData))



0.9423076923076923


## **Task 6 (35 Marks):**

Consider a modification to your algorithm that would improve the accuracy of the algorithm developed in Task 3

_Marks will be awarded based on how interesting the proposed improvement is. Please provide a short text describing what you intend to do and why. Full marks for this section may be obtained without an implementation, but an implementation is preferred._


# The modified algorithm is thus:
 1. Get the number of all possible bigrams in the training dataset. Let us name it countOfAllPossibleBigrams
 2. repeat for every word in the sentence:
     
     if(the incorrect word is neither the first or the last word and is not the only word in the sentence):
         
       a. Get the bigram frequency of the candidate (replacement for the incorrect word) with the word preceding it, add 1 to it, and divide it by countOfAllPossibleBigrams.
       
       b. Get the bigram frequency of the candidate with the word following it, add 1 to it, and divide it by countOfAllPossibleBigrams.
       
       c. Multiply the contents of 2a. and 2b.
       
       d. if( the probability == 0):
            Take care of zero probability by giving the probability as 0.1/countOfAllPossibleBigrams. This is done so as to penalize bigrams which do not occur in the corpus.
     
     if(the incorrect word is the first word):
       
       e. calculate the bigram frequency of the candidate with the word following it, add 1 to it , and divide it by countOfAllPossibleBigrams.
       
       f. GOTO 2d.
     
     if(the incorrect word is the last word):
       
       g. calculate the bigram frequency of the candidate with the word preceding it, add 1 to it , and divide it by countOfAllPossibleBigrams.
       
       h. GOTO 2d.
       
     if(there is only one word in the sentence):
       
       i. return the first candidate in the list.
 3. Return the corrected sentence.

## **Task 7 (5 Marks):**

Repeat the evaluation of your new algorithm and show that it outperforms the algorithm from Task 3

In [19]:
#First we find the possible number of bigrams in the trainingData
counting=[]
for i in range(len(trainingData)):
    length=len(trainingData[i][1])
    counting.append(length-1)
sum(counting)
#counting gives us the sum of all possible consecutive bigrams

    

19481

In [24]:
import nltk
from itertools import chain
import math

# make a dictionary of all correct words from holbrook
dictionary1 = []
for i in range(len(COR)):
    dictionary1.append([x for x in COR[i]])

dictionary1 = list(chain.from_iterable(dictionary1))

def correct_v2(sentence):
    correctSentence = []
    for i in range(len(sentence)):
        if(sentence[i] in list(chain.from_iterable(dictionary1))):
            correctSentence.append(sentence[i])
        else:
            get_cand = list((get_candidates(sentence[i])))    
            if(i < (len(sentence)-1) and i > 0):
                
                prefix = sentence[i-1]
                suffix = sentence[i+1]
                conditional_probability = []
                for a in range(0,len(get_cand)): 
                    #Here we calculate probabilities as follows
                    #Get the bigram frequency of the candition and the word preceding it and divide it by the total
                    #possible bigrams in the correct section of the traning data
                    #Multiply it by the bigram frequency of the word followed by the candidate and
                    #divide it by the sum(counting) i.e. the number of all possible bigrams  
                    x=((bigram(prefix + " " + get_cand[a]) +1)/sum(counting))*((bigram(get_cand[a] + " " + suffix)+1)/sum(counting))
                    if(x>0):
                        conditional_probability.append(((bigram(prefix + " " + get_cand[a]) +1)/sum(counting))*((bigram(get_cand[a] + " " + suffix)+1)/sum(counting)))
                    else:
                        #This appends 1/sum(counting) in case of zero probability
                        conditional_probability.append(0.01/sum(counting))
                highest_prob_ind = conditional_probability.index(max(conditional_probability))
                correctSentence.append(get_cand[highest_prob_ind])
            else:
                index = sentence.index(sentence[i])
                if(index == 0 and len(sentence)>1):
                    suffix = sentence[i+1]
                    conditional_probability = []
                    for a in range(0,len(get_cand)):
                        #if it is the first word which is incorrect in a sentence
                        #get its bigram frequency with the word following it and divide it by all possible ones.
                        y=(bigram(get_cand[a] + " " + suffix))
                        x=y+1/sum(counting)
                        if(x>0):
                            conditional_probability.append((bigram(get_cand[a] + " " + suffix)+1)/sum(counting))
                        else:
                            conditional_probability.append(0.01/sum(counting))
                    correctSentence.append(get_cand[conditional_probability.index(max(conditional_probability))])
                elif(len(sentence)>1):
                    prefix = sentence[i-1]
                    cdf = []
                    for a in range(0,len(get_cand)):
                        #get bigram with the word preceding it 
                        #and divide by the number of all possible occurences of bigrams
                        y=bigram(prefix + " " + get_cand[a])
                        x=y+1/sum(counting)
                        if(x>0):
                            cdf.append((bigram(prefix + " " + get_cand[a])+1)/sum(counting))
                        else:
                            cdf.append(0.01/sum(counting))
                    correctSentence.append(get_cand[cdf.index(max(cdf))])
                else:
                    #if the sentence just consists of a word (which itself is incorrect)
                    #select the first candidate as the changed word
                    x=list((get_candidates(sentence[i])))[0]
                    correctSentence.append(x)
            
    return correctSentence



        
correct_v2(['my', 'dad', 'works', 'at', 'meltn'])

['my', 'dad', 'works', 'at', 'melton']

In [25]:
#Checking the accuracy of the algorithm modified
def accuracy_v2(test):
    count=0
    totalCount=0
    for i in range(len(dictionary)):
        for j in range(len(dictionary[i][2])):
            if(len(dictionary[i][2])> 0 and i<99):
                m=correct_v2(list(chain.from_iterable(test[i][0])))
                if(m[j]==dictionary[i][1][j][0]):
                    count=count+1
                    totalCount=totalCount+1
                else:
                    totalCount=totalCount+1
    return(count/totalCount)

print(accuracy_v2(testingData))

0.9519230769230769


The accuracy by the new method has increased by about 1%

# ASHFAQUE AZAD 